In [8]:
import pandas as pd
from konlpy.tag import Komoran, Kkma, Okt
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from math import log
from operator import itemgetter
from pymongo import MongoClient

In [20]:
dataset = pd.read_csv('D:/LikeLion/Code/Project2/Data/spell_check_label.csv',encoding='utf-8')

In [14]:
with open('D:/LikeLion/Code/Project2/Data/kor_stop.txt', 'r', encoding='utf-8') as f:
    list_file = f.readlines()
stopwords = [line.rstrip('\n') for line in list_file] 

In [23]:
hotel_name = list(dataset.groupby('hotelName').count().index)
groups = dataset.groupby(dataset.hotelName)

In [24]:
for i, name in enumerate(hotel_name):
    globals()['df_{}'.format(i)] = groups.get_group(name)

In [53]:
df_0_neg = df_0[df_0['label']==0].copy()

In [54]:
df_0_neg['fixed'] = df_0_neg['fixed'].apply(lambda x: re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",x))

In [55]:
document_count = len(df_0_neg)

In [28]:
komoran = Komoran(userdic='D:/LikeLion/Code/Project2/Data/userdic.txt')
kkma = Kkma()
okt = Okt()

In [56]:
def tokenizer(text):
    # josa = ['JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC', 'EP', 'EF', 'EC', 'ETN', 'ETM', 'XPN', 'XSN', 'XSV', 'XSA', 'XR']
    pos_tag = ['NNG', 'NNP', 'NNB', 'NP', 'NR', 'VA', 'MM', 'MAG', 'XR']
    tokens_pos = komoran.pos(text)
    tokens_word = [] 
    for tag in tokens_pos:
        if tag[1] in pos_tag:
            if tag[0] not in stopwords:
                tokens_word.append(tag[0])
    return re.sub('\.','',' '.join(tokens_word))

In [57]:
df_0_neg['fixed'] = df_0_neg['fixed'].apply(lambda x: tokenizer(x))

In [58]:
documents = df_0_neg['fixed'].to_list()

In [32]:
# def tokenize(sent):
#     words = komoran.pos(sent, join=True)
#     words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w or '/VX' in w or '/VC' in w or '/MM' in w or '/MA' in w or '/IC' in w
#                                   or '/E' in w or '/XP' in w or '/XS' in w or '/XR' in w or '/SF' in w or '/SP' in w or '/SS' in w or '/SE' in w or '/SO' in w
#                                   or '/SL' in w or '/SW' in w or '/NF' in w or '/NV' in w or '/SN' in w )]
#     return words

In [59]:
def get_term_frequency(document, word_dict=None):
    if word_dict is None:
        word_dict = {}
    words = document.split()

    for w in words:
        word_dict[w] = 1 + (0 if word_dict.get(w) is None else word_dict[w])

    return pd.Series(word_dict, dtype='float64').sort_values(ascending=False)

In [34]:
def get_document_frequency(documents):
    dicts = []
    vocab = set([])
    df = {}

    for d in documents:
        tf = get_term_frequency(d)
        dicts += [tf]
        vocab = vocab | set(tf.keys())

    for v in list(vocab):
        df[v] = 0
        for dict_d in dicts:
            if dict_d.get(v) is not None:
                df[v] += 1
    
    return pd.Series(df, name='df', dtype='float64').sort_values(ascending=False)

In [60]:
def get_tf(docs):
    vocab = {}
    tfs = []
    for d in docs:
        vocab = get_term_frequency(d, vocab)
        tfs += [get_term_frequency(d)]

    stats = []

    for word, freq in vocab.items():
        tf_v = []
        for idx in range(len(docs)):
            if tfs[idx].get(word) is not None:
                tf_v += [tfs[idx][word]]
            
            else:
                tf_v += [0]

        stats.append((word, freq, *tf_v))

    column_name = ['word','totalFrequency']

    for i in range(1, len(docs)+1):
        column_name.append('document'+str(i))

    return pd.DataFrame(stats, columns=column_name).sort_values('totalFrequency', ascending=False)

In [36]:
# def get_tfidf(docs):
#     vocab = {}
#     tfs = []
#     for d in docs:
#         vocab = get_term_frequency(d, vocab)
#         tfs += [get_term_frequency(d)]
    
#     df = get_document_frequency(docs)

#     stats = []

#     for word, freq in vocab.items():
#         tfidfs = []
#         for idx in range(len(docs)):
#             if tfs[idx].get(word) is not None:
#                 tfidfs += [tfs[idx][word] * np.log(len(docs) / df[word])]
            
#             else:
#                 tfidfs += [0]

#         stats.append((word, freq, *tfidfs, max(tfidfs)))
    
#     column_name = ['word','totalFrequency']

#     for i in range(1, len(docs)+1):
#         column_name.append('document'+str(i))
    
#     column_name.append('max')

#     return pd.DataFrame(stats, columns=column_name).sort_values('max', ascending=False)

In [61]:
tf_matrix = get_tf(documents)
df_matrix = get_document_frequency(documents)

In [62]:
df_matrix

먼지    29.0
청소    28.0
방음    26.0
침대    25.0
객실    22.0
      ... 
물이     1.0
착용     1.0
사진     1.0
사인     1.0
미술     1.0
Name: df, Length: 770, dtype: float64

In [63]:
tf_matrix

,word,totalFrequency,document1,document2,document3,document4,document5,document6,document7,document8,...,document415,document416,document417,document418,document419,document420,document421,document422,document423,document424
0,청소,30.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,먼지,30.0,0,0.0,1.0,0.0,0.0,0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,침대,29.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,방음,27.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,객실,22.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,공사장,1.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
467,정비,1.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
466,통,1.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
465,상당히,1.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def get_ntf(matrix):
    max_btf = max(matrix['totalFrequency'])
    total_btf = sum(matrix['totalFrequency'])

    col_names = list(matrix.columns)[2:]

    matrix['ntf1'] = matrix['totalFrequency'].apply(lambda x: x/max_btf)

    matrix_ntf2 = matrix[col_names].copy()

    matrix_ntf2 = matrix_ntf2.apply(lambda x: x/total_btf , axis=1)
    
    matrix_ntf2['ntf2'] = matrix_ntf2.apply(sum, axis=1)

    matrix['ntf2'] = matrix_ntf2['ntf2']

    return matrix[['word','totalFrequency','ntf1','ntf2']]

In [64]:
ntf_matrix = get_ntf(tf_matrix)

In [65]:
def get_ntf_idf(ntf, df):

    ntf = ntf.set_index('word')
    ntf_idf = pd.concat([ntf, df], axis=1)


    def get_btfidf(scores):
        global document_count
        return (np.log(scores['totalFrequency']) + 1.0) * np.log(document_count/scores['df'])

    def get_ntf1idf(scores):
        global document_count
        return (np.log(scores['ntf1']) + 1.0) * np.log(document_count/scores['df'])

    def get_ntf2idf(scores):
        global document_count
        return (np.log(scores['ntf2']) + 1.0) * np.log(document_count/scores['df'])

    
    ntf_idf['btf_idf'] = ntf_idf.apply(get_btfidf, axis=1)
    ntf_idf['ntf1_idf'] = ntf_idf.apply(get_ntf1idf, axis=1)
    ntf_idf['ntf2_idf'] = ntf_idf.apply(get_ntf2idf, axis=1)

    btf_rank = ntf_idf['btf_idf'].sort_values(ascending=False)
    ntf1_rank = ntf_idf['ntf1_idf'].sort_values(ascending=False)
    ntf2_rank = ntf_idf['ntf2_idf'].sort_values(ascending=False)

    return btf_rank, ntf1_rank, ntf2_rank

In [66]:
btf_rank , ntf1_rank, ntf2_rank = get_ntf_idf(ntf_matrix, df_matrix)

In [67]:
# btf_rank.to_csv('../../Data/btf.csv', encoding='utf-8-sig')
# ntf1_rank.to_csv('../../Data/ntf1.csv',encoding='utf-8-sig')
# ntf2_rank.to_csv('../../Data/ntf2.csv', encoding='utf-8-sig')